In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:

import re
import pandas as pd

# Sample data for testing
SAMPLE_DATA = """- 2025-07-18 12:43:08
  Username: gianni
  Message: Eccoti i dati da escludere questa settimana: A13-1-9 product B escludere pdu16 e 18

- 2025-04-07 06:51:35
  Username: maria
  Message: riusciresti a dirmi qual'è la peak performance per referenza? Grazie mille

- 2025-03-06 07:29:37
  Username: maria
  Message: Ciao serena, in week 9 non mi tornano i dati di product 6 e product a1. E' possibile che non sono stati filtrati i dati quando le bilance non erano calibrate?

- 2025-02-19 13:33:59
  Username: gianni
  Message: escludere A10S1L1, A10S2L1 (dopo production start il 3), il resto mi sembra a posto

- 2025-06-20 12:34:42
  Username: gianni
  Message: product 7 A1-2-6 considerare solo pdu da 1 a 6, Misti A11-2-2 considerare solo pdu da 1 a 6, A11S2L1 escludere pdu1-2-3"""

def parse_messages(text):
    """Parse messages from text"""
    messages = []
    pattern = r'- (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})\n\s+Username: (.*?)\n\s+Message: (.*?)(?=\n- \d{4}|\Z)'

    for match in re.finditer(pattern, text, re.DOTALL):
        messages.append({
            'timestamp': match.group(1),
            'user': match.group(2),
            'message': match.group(3).strip()
        })

    return messages

def analyze_message(msg):
    """Extract information from a single message"""
    msg_text = msg['message']
    msg_lower = msg_text.lower()

    # Determine request type
    if any(word in msg_lower for word in ['escludere', 'exclude', 'togliere']):
        request_type = 'exclusion'
    elif any(word in msg_lower for word in ['outcome', 'resa', 'peak', 'performance']):
        request_type = 'analysis'
    else:
        request_type = 'general'

    # Extract positions
    positions = re.findall(r'A\d{1,2}[-S]\d[-L]\d{1,2}', msg_text)

    # Extract PDUs
    pdus = re.findall(r'pdu\s*(\d+)', msg_text, re.IGNORECASE)

    # Extract weeks
    weeks = re.findall(r'week\s*(\d+)|settimana\s*(\d+)', msg_text, re.IGNORECASE)
    weeks = [w[0] or w[1] for w in weeks if w[0] or w[1]]

    # Extract varieties
    varieties = []
    variety_list = ['product 6', 'product 7', 'product g2', 'product g3',
                    'product m', 'product a1', 'product a2', 'product b']
    for v in variety_list:
        if v in msg_lower:
            varieties.append(v)

    return {
        'timestamp': msg['timestamp'],
        'user': msg['user'],
        'request_type': request_type,
        'message_preview': msg_text[:80],
        'positions': positions,
        'pdus': pdus,
        'weeks': weeks,
        'varieties': varieties
    }

def generate_simple_sql(analysis):
    """Generate SQL based on analysis"""
    sql_parts = []

    if analysis['request_type'] == 'exclusion':
        sql_parts.append("-- Exclusion Query")
        sql_parts.append("SELECT * FROM outcome_per_ordercode")
        sql_parts.append("WHERE test = FALSE")

        for pos in analysis['positions']:
            sql_parts.append(f"  AND position != '{pos}'")

        for pdu in analysis['pdus']:
            sql_parts.append(f"  AND pdu_number != {pdu}")

    elif analysis['request_type'] == 'analysis':
        sql_parts.append("-- Analysis Query")
        sql_parts.append("SELECT variety, week_number,")
        sql_parts.append("  AVG(outcome_kg) as avg_outcome,")
        sql_parts.append("  MAX(outcome_kg) as peak_outcome")
        sql_parts.append("FROM outcome_per_ordercode")
        sql_parts.append("WHERE test = FALSE")

        if analysis['varieties']:
            variety_list = "', '".join(analysis['varieties'])
            sql_parts.append(f"  AND variety IN ('{variety_list}')")

        if analysis['weeks']:
            week_list = ", ".join(analysis['weeks'])
            sql_parts.append(f"  AND week_number IN ({week_list})")

        sql_parts.append("GROUP BY variety, week_number;")

    else:
        sql_parts.append("-- General Query")
        sql_parts.append("SELECT * FROM outcome_per_ordercode LIMIT 10;")

    return "\n".join(sql_parts)

# Main execution
print("🌱 VERTICAL FARM MESSAGE ANALYZER")
print("="*60)

# Choose input method
print("\n1. Use sample data")
print("2. Load from Google Drive")

choice = input("Enter choice (1 or 2): ").strip()

if choice == '2':
    from google.colab import drive
    drive.mount('/content/drive')
    file_path = input("Enter file path (e.g., /content/drive/MyDrive/messages.txt): ")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except:
        print("Error loading file, using sample data")
        text = SAMPLE_DATA
else:
    text = SAMPLE_DATA

# Process messages
print("\n📊 Processing messages...")
messages = parse_messages(text)
print(f"Found {len(messages)} messages")

# Analyze each message
results = []
sql_queries = []

for msg in messages:
    analysis = analyze_message(msg)
    results.append({
        'timestamp': analysis['timestamp'],
        'user': analysis['user'],
        'type': analysis['request_type'],
        'message': analysis['message_preview'],
        'positions': ', '.join(analysis['positions']),
        'pdus': ', '.join(analysis['pdus']),
        'weeks': ', '.join(analysis['weeks']),
        'varieties': ', '.join(analysis['varieties'])
    })

    if analysis['request_type'] in ['exclusion', 'analysis']:
        sql = generate_simple_sql(analysis)
        sql_queries.append(f"-- {analysis['user']} ({analysis['timestamp']})\n{sql}")

# Create DataFrame and save
df = pd.DataFrame(results)
df.to_csv('analysis_results.csv', index=False)
print(f"\n✅ Saved {len(df)} results to analysis_results.csv")

# Save SQL queries
if sql_queries:
    with open('queries.sql', 'w') as f:
        f.write("\n\n".join(sql_queries))
    print(f"✅ Saved {len(sql_queries)} SQL queries to queries.sql")

# Show summary
print("\n📈 Summary:")
print(f"Users: {df['user'].value_counts().to_dict()}")
print(f"Request types: {df['type'].value_counts().to_dict()}")

print("\n✅ Complete! Check the generated files.")

# Display sample results
print("\nSample results:")
print(df.head())

🌱 VERTICAL FARM MESSAGE ANALYZER

1. Use sample data
2. Load from Google Drive

📊 Processing messages...
Found 5 messages

✅ Saved 5 results to analysis_results.csv
✅ Saved 4 SQL queries to queries.sql

📈 Summary:
Users: {'gianni': 3, 'maria': 2}
Request types: {'exclusion': 3, 'analysis': 1, 'general': 1}

✅ Complete! Check the generated files.

Sample results:
             timestamp    user       type  \
0  2025-07-18 12:43:08  gianni  exclusion   
1  2025-04-07 06:51:35   maria   analysis   
2  2025-03-06 07:29:37   maria    general   
3  2025-02-19 13:33:59  gianni  exclusion   
4  2025-06-20 12:34:42  gianni  exclusion   

                                             message  \
0  Eccoti i dati da escludere questa settimana: A...   
1  riusciresti a dirmi qual'è la peak performance...   
2  Ciao serena, in week 9 non mi tornano i dati d...   
3  escludere A10S1L1, A10S2L1 (dopo production st...   
4  product 7 A1-2-6 considerare solo pdu da 1 a 6...   

                  positions